In [86]:
import numpy_financial as npf
from dataclasses import dataclass
import pandas as pd
import numpy as np

In [87]:
def calculate_chit_fund_analysis():
    # --- SCHEME PARAMETERS ---
    total_chit_value = 700000
    total_members = 14
    duration_years = 7
    frequency_per_year = 2  # Semi-annual
    
    # Full installment amount before dividend deduction
    base_installment = total_chit_value / total_members  # 50,000
    
    # --- HYPOTHETICAL DIVIDEND SCHEDULE ---
    # In a real scenario, these are the actual dividends declared in every auction.
    # We are simulating varying dividends (high at start, low at end).
    # Format: [Period 1 dividend, Period 2 dividend, ... Period 14 dividend]
    dividends = [
        12000, 11500, 11000, 10000, 9000, 8000, 
        7000, 6000, 5000, 4000, 3000, 2000, 1000, 0
    ]
    
    # Calculate the actual payment schedule for an Investor (Saver)
    # Investor pays (Base Installment - Dividend) every period
    investor_payments = [(base_installment - div) for div in dividends]

    print(f"{'='*40}")
    print(f"CHIT FUND SCHEME ANALYSIS")
    print(f"{'='*40}")
    print(f"Base Installment: ₹{base_installment:,.2f}")
    print(f"Total Duration: {duration_years} Years ({total_members} Periods)")
    print("-" * 40)

    # ==========================================
    # SCENARIO 1: THE INVESTOR (SAVER)
    # ==========================================
    # Logic: Future Value of Uneven Cash Flows / IRR approach
    
    # Step 1: Create Cash Flow Stream
    # Outflows are negative, Inflow (Maturity) is positive
    investor_cash_flows = [-payment for payment in investor_payments]
    
    # At the end of the last period, the investor receives the pot
    # Note: Usually the last payment is adjusted or paid and then pot received. 
    # Here we assume pot is received after the cycle.
    investor_cash_flows.append(total_chit_value) 
    
    # Step 2: Calculate Absolute Gain
    total_invested = sum(investor_payments)
    net_gain = total_chit_value - total_invested
    
    # Step 3: Calculate ROI (IRR)
    # The irr function finds the rate that makes NPV = 0
    periodic_irr = npf.irr(investor_cash_flows)
    
    # Annualize the rate: (1 + r)^2 - 1
    annual_roi = ((1 + periodic_irr) ** frequency_per_year) - 1

    print(f"\n1. INVESTOR (SAVER) ANALYSIS")
    print(f"Total Amount Invested: ₹{total_invested:,.2f}")
    print(f"Maturity Amount Received: ₹{total_chit_value:,.2f}")
    print(f"Absolute Net Gain: ₹{net_gain:,.2f}")
    print(f"Yearly ROI (XIRR/Effective): {annual_roi:.2%}")

    # ==========================================
    # SCENARIO 2: THE BID WINNER (BORROWER)
    # ==========================================
    # Logic: Implicit Interest Rate / Cost of Capital
    
    # Assumption: User wins in Period 2 (index 1)
    winning_period_index = 1 
    
    # Assumption: Prize Money (Total Chit - Bid Amount)
    # Let's assume the bid amount was high early on, say ₹1,40,000 discount
    prize_money = 560000 
    
    borrower_cash_flows = []
    
    # Step 1: Cash Flows BEFORE winning (Variable payments)
    # These are outflows (-)
    for i in range(winning_period_index):
        borrower_cash_flows.append(-investor_payments[i])
        
    # Step 2: Cash Flow ON winning (Net Inflow)
    # Inflow = Prize Money - Current Period Payment
    net_inflow_at_win = prize_money - investor_payments[winning_period_index]
    borrower_cash_flows.append(net_inflow_at_win)
    
    # Step 3: Cash Flows AFTER winning (Fixed Full Payments)
    # Winner pays full 50,000 from next bid onwards (no dividends)
    remaining_periods = total_members - (winning_period_index + 1)
    for _ in range(remaining_periods):
        borrower_cash_flows.append(-base_installment)
        
    # Step 4: Calculate Net Interest Paid (Absolute)
    total_paid_pre_win = sum(investor_payments[:winning_period_index+1])
    total_paid_post_win = base_installment * remaining_periods
    total_outflow_borrower = total_paid_pre_win + total_paid_post_win
    net_interest_cost = total_outflow_borrower - prize_money

    # Step 5: Calculate Implicit Interest Rate (Cost of Borrowing)
    borrower_periodic_rate = npf.irr(borrower_cash_flows)
    borrower_annual_rate = ((1 + borrower_periodic_rate) ** frequency_per_year) - 1

    print(f"\n2. BID WINNER (BORROWER) ANALYSIS")
    print(f"Won in Period: {winning_period_index + 1}")
    print(f"Prize Money Received: ₹{prize_money:,.2f}")
    print(f"Total Repayment (Pre + Post Win): ₹{total_outflow_borrower:,.2f}")
    print(f"Net Interest Cost (Absolute): ₹{net_interest_cost:,.2f}")
    print(f"Implicit Interest Rate (Yearly Cost of Debt): {borrower_annual_rate:.2%}")



In [88]:
r=0.06/2
p=10*2
50000*(1+r)**p

90305.56173347073

In [4]:
p=2
25000/(1+0.1)**p

20661.157024793385

In [89]:
total_installments = 14
current_installment_number=4
full_chit_value = 700000
chit_frequency_per_year=2
previous_installments = [42000,40000,40000,43000]
bid_amount = 100000


def get_prize_amount(full_chit_value, bid_amount):
    return full_chit_value - bid_amount

def create_cashflows(previous_installments, prize_amount, winner_installment_amount, current_installments_number, total_installments):
    future_winner_installments=[-winner_installment_amount]* (total_installments - current_installments_number)
    cashflows = [i * -1 for i in previous_installments] + [prize_amount] + future_winner_installments
    return cashflows


def calculate_annual_irr(cashflows, chit_frequency_per_year):
    irr_per_period=npf.irr(cashflows)
    annual_irr=((1 + irr_per_period) ** chit_frequency_per_year) - 1
    return annual_irr


prize_amount = get_prize_amount(full_chit_value, bid_amount)
cashflows = create_cashflows(
    previous_installments,
    prize_amount,
    50000,
    current_installment_number,
    total_installments,
)
annual_irr = calculate_annual_irr(cashflows, chit_frequency_per_year)
print(f"Annual IRR for Bid Winner: {annual_irr:.2%}")

Annual IRR for Bid Winner: 6.60%


In [90]:
total_installments = 14
current_installment_number = 4
full_chit_value = 700000
chit_frequency_per_year = 2
previous_installments = [42000, 40000, 40000, 43000]
bid_amount = 100000


def get_prize_amount(full_chit_value, bid_amount):
    return full_chit_value - bid_amount


def create_cashflows(
    previous_installments,
    prize_amount,
    winner_installment_amount,
    current_installments_number,
    total_installments,
):
    future_winner_installments = [-winner_installment_amount] * (
        total_installments - current_installments_number
    )
    cashflows = (
        [i * -1 for i in previous_installments]
        + [prize_amount]
        + future_winner_installments
    )
    return cashflows


def calculate_annual_irr(cashflows, chit_frequency_per_year):
    irr_per_period = npf.irr(cashflows)
    annual_irr = ((1 + irr_per_period) ** chit_frequency_per_year) - 1
    return annual_irr

def analyze_bid_scenarios(
    total_installments,
    current_installment_number,
    full_chit_value,
    chit_frequency_per_year,
    previous_installments,
    bid_amounts_to_test=None,
    winner_installment_amount=50000
):
    """
    Analyze multiple bid amount scenarios to compare IRR and total costs.
    
    Parameters:
    - total_installments: Total number of installments in the chit
    - current_installment_number: The installment number when winning
    - full_chit_value: Total chit fund value
    - chit_frequency_per_year: Payment frequency (e.g., 2 for semi-annual)
    - previous_installments: List of previous installment amounts paid
    - bid_amounts_to_test: List of bid amounts to analyze (optional)
    - winner_installment_amount: Fixed amount winner pays after winning
    
    Returns:
    - DataFrame with scenario analysis results
    """
    
    if bid_amounts_to_test is None:
        # Default: Test bid amounts from 50k to 200k in 10k increments
        bid_amounts_to_test = range(50000, 210000, 10000)
    
    results = []
    
    for bid_amount in bid_amounts_to_test:
        # Reuse existing functions
        prize_amount = get_prize_amount(full_chit_value, bid_amount)
        cashflows = create_cashflows(
            previous_installments,
            prize_amount,
            winner_installment_amount,
            current_installment_number,
            total_installments,
        )
        
        try:
            annual_irr = calculate_annual_irr(cashflows, chit_frequency_per_year)
            
            # Calculate total repayment
            total_paid_before = sum(previous_installments)
            total_paid_after = winner_installment_amount * (total_installments - current_installment_number)
            total_repayment = total_paid_before + total_paid_after
            
            # Calculate net interest cost
            net_interest_cost = total_repayment - prize_amount
            
            results.append({
                'Bid Amount': bid_amount,
                'Prize Received': prize_amount,
                'Total Repayment': total_repayment,
                'Net Interest Cost': net_interest_cost,
                'Annual IRR (%)': annual_irr * 100,
                'Effective Interest (%)': (net_interest_cost / prize_amount) * 100
            })
        except:
            # Handle cases where IRR cannot be calculated
            results.append({
                'Bid Amount': bid_amount,
                'Prize Received': prize_amount,
                'Total Repayment': None,
                'Net Interest Cost': None,
                'Annual IRR (%)': None,
                'Effective Interest (%)': None
            })
    
    df = pd.DataFrame(results)
    return df




# Example usage with current scenario
print("=" * 80)
print("BID AMOUNT SCENARIO ANALYSIS")
print("=" * 80)

scenario_results = analyze_bid_scenarios(
    total_installments=total_installments,
    current_installment_number=current_installment_number,
    full_chit_value=full_chit_value,
    chit_frequency_per_year=chit_frequency_per_year,
    previous_installments=previous_installments,
    bid_amounts_to_test=range(50000, 210000, 10000),
    winner_installment_amount=50000
)

# Display formatted results
pd.options.display.float_format = '{:,.2f}'.format
print(scenario_results.to_string(index=False))

# Find optimal bid (lowest IRR)
optimal_row = scenario_results.loc[scenario_results['Annual IRR (%)'].idxmin()]
print("\n" + "=" * 80)
print("OPTIMAL BID STRATEGY")
print("=" * 80)
print(f"Bid Amount: ₹{optimal_row['Bid Amount']:,.0f}")
print(f"Prize Received: ₹{optimal_row['Prize Received']:,.0f}")
print(f"Annual IRR: {optimal_row['Annual IRR (%)']:.2f}%")
print(f"Net Interest Cost: ₹{optimal_row['Net Interest Cost']:,.0f}")



BID AMOUNT SCENARIO ANALYSIS
 Bid Amount  Prize Received  Total Repayment  Net Interest Cost  Annual IRR (%)  Effective Interest (%)
      50000          650000           665000              15000            1.33                    2.31
      60000          640000           665000              25000            2.27                    3.91
      70000          630000           665000              35000            3.26                    5.56
      80000          620000           665000              45000            4.31                    7.26
      90000          610000           665000              55000            5.42                    9.02
     100000          600000           665000              65000            6.60                   10.83
     110000          590000           665000              75000            7.85                   12.71
     120000          580000           665000              85000            9.20                   14.66
     130000          570000        

In [74]:
future_winner_installments


[-50000, -50000, -50000, -50000, -50000, -50000, -50000, -50000, -50000]

In [70]:
cashflows=[-40000 for i in range(4)] + [690000] + [-50000 for i in range(10)]
cashflows


[-40000,
 -40000,
 -40000,
 -40000,
 690000,
 -50000,
 -50000,
 -50000,
 -50000,
 -50000,
 -50000,
 -50000,
 -50000,
 -50000,
 -50000]

In [40]:
# Add this to your existing IRR calculation cell

irr_result = npf.irr(cashflows)
annual_irr = (1 + irr_result) ** 2 - 1

# NEW: Calculate NPV at different discount rates
print("\n" + "=" * 50)
print("NPV ANALYSIS (Complementary to IRR)")
print("=" * 50)

# At IRR, NPV should be ~0 (verification)
npv_at_irr = npf.npv(irr_result, cashflows)
print(f"NPV at IRR ({irr_result:.2%}):        ₹{npv_at_irr:,.2f} (should be ≈ 0)")

# At market rate (e.g., 10% annual = 4.88% semi-annual)
market_rate = (1.10**0.5) - 1  # Semi-annual equivalent of 10% annual
npv_at_market = npf.npv(market_rate, cashflows)
print(f"NPV at market rate (10% annual):   ₹{npv_at_market:,.2f}")

if npv_at_market < 0:
    print("  → Deal is WORSE than market rate (you're overpaying)")
else:
    print("  → Deal is BETTER than market rate (you're underpaying)")

# Decision rule
print("\n" + "-" * 50)
print("DECISION RULE:")
print("-" * 50)
if annual_irr > 0.10:
    print(f"✗ IRR of {annual_irr:.2%} EXCEEDS your 10% opportunity cost")
    print("  → NOT worth borrowing at this IRR")
else:
    print(f"✓ IRR of {annual_irr:.2%} is BELOW your 10% opportunity cost")
    print("  → Worth considering if you need the cash now")



NPV ANALYSIS (Complementary to IRR)
NPV at IRR (-4.43%):        ₹0.00 (should be ≈ 0)
NPV at market rate (10% annual):   ₹182,785.28
  → Deal is BETTER than market rate (you're underpaying)

--------------------------------------------------
DECISION RULE:
--------------------------------------------------
✓ IRR of -8.66% is BELOW your 10% opportunity cost
  → Worth considering if you need the cash now


In [ ]:
import numpy as np
c0=560000
fv=50000
remaining_period=14
for intrest in np.arange(-5, 9, 0.5):
    r=(intrest/100)
    pv = sum(
        [
            fv / (1 + r) ** (i + 1) if i >= 4 else (fv - 10000) / (1 + r) ** (i + 1)
            for i in range(remaining_period)
        ]
    )
    print(f"At interest rate {intrest}%, Present Value is: ₹{pv:,.2f}")
    if round(pv)==0:
        print(f"At interest rate {intrest}%, Present Value is: ₹{pv:,.2f}")
        break

In [62]:
for i in range(5):
    print((1 + 0.1) ** (i+1))
    print(25000 / (1 + 0.1) ** (i+1))

1.1
22727.272727272724
1.2100000000000002
20661.157024793385
1.3310000000000004
18782.87002253944
1.4641000000000004
17075.336384126764
1.6105100000000006
15523.033076478874


In [47]:
calculate_chit_fund_analysis()

CHIT FUND SCHEME ANALYSIS
Base Installment: ₹50,000.00
Total Duration: 7 Years (14 Periods)
----------------------------------------
Yearly ROI (XIRR/Effective): 3.82%

1. INVESTOR (SAVER) ANALYSIS
Total Amount Invested: ₹610,500.00
Maturity Amount Received: ₹700,000.00
Absolute Net Gain: ₹89,500.00
Yearly ROI (XIRR/Effective): 3.82%

2. BID WINNER (BORROWER) ANALYSIS
Won in Period: 2
Prize Money Received: ₹560,000.00
Total Repayment (Pre + Post Win): ₹676,500.00
Net Interest Cost (Absolute): ₹116,500.00
Implicit Interest Rate (Yearly Cost of Debt): 7.20%


In [108]:
import numpy as np

C = 700000
instalments = 14
installment_amt = C / instalments
discounts = np.arange(0.05, 0.45, 0.01) * C
best = None

for D in discounts:
    cashflows = [-(installment_amt)] * instalments
    cashflows[0] = C - D  # winning amount
    irr = npf.irr(cashflows)
    annual_rate = (1 + irr) ** 2 - 1
    if best is None or annual_rate < best[1]:
        best = (D, annual_rate)

print(f"Optimal Discount: {best[0]:.0f}, Interest Rate: {best[1] * 100:.2f}%")


Optimal Discount: 35000, Interest Rate: -0.65%


In [9]:
50000*0.1*0.

2000.0

In [1]:
import random
from statistics import mean

TOTAL_CHIT = 700_000
MEMBERS = 14
PERIODS = MEMBERS  # one winner per bid
PER_PERIOD = TOTAL_CHIT // MEMBERS  # 50,000
TARGET_BID_INDEX = 6  # 1-based indexing for human, we'll use 0-based in code


def simulate_once(d6_discount: int, seed: int | None = None) -> float:
    """
    Simulate one scenario for a member who wins at bid 6 with discount d6_discount.
    Other discounts for bids 1-5 are random in [7000, 10000].
    Returns interest percent for the target member.
    """
    if seed is not None:
        random.seed(seed)

    # Random discounts for bids 1-5, single candidate per bid
    discounts = [
        random.randint(7000, 10000) for _ in range(TARGET_BID_INDEX - 1)
    ]  # bids 1..5
    discounts.append(d6_discount)  # bid 6
    # For bids 7..14, we can continue random sampling within same range or set to a default.
    # The user asked to simulate randomly only till the 6th bid; beyond that, values don't affect
    # the winner (since the winner pays full and receives no dividends). We'll still create placeholders.
    discounts.extend([0] * (PERIODS - len(discounts)))

    # Track target member cashflows
    total_payments = 0
    total_received = 0

    # Track per-bid accrued future-interest-shares for non-winner periods before bid 6
    # We accumulate reductions that apply to each future payment until bid 6.
    # After winning at 6, our payments are full (no reductions).
    future_interest_shares = [
        0.0
    ] * PERIODS  # per future bid deduction (for non-winner periods)

    # Iterate bids
    for bid_idx in range(PERIODS):
        discount = discounts[bid_idx]

        # Everyone pays for the current bid
        payment_this_bid = PER_PERIOD

        if bid_idx < TARGET_BID_INDEX - 1:
            # Before winning (bids 0..4): target is a non-winner and gets future interest shares
            total_payments += payment_this_bid
            # Compute interest share to be spread over remaining bids after this one
            remaining_bids = PERIODS - (bid_idx + 1)
            if remaining_bids > 0 and discount > 0:
                share = discount / remaining_bids
                # Apply this share as a deduction to each future bid payment until we win
                for future_idx in range(bid_idx + 1, TARGET_BID_INDEX - 1):
                    future_interest_shares[future_idx] += share

        elif bid_idx == TARGET_BID_INDEX - 1:
            # Winning bid (bid 6): target receives pot minus discount, also pays this bid
            total_payments += payment_this_bid
            total_received += TOTAL_CHIT - discount
            # From next bid onward, no dividends are received; they pay full amount
            # So we do not add any future_interest_shares after this point.

        else:
            # After winning (bids 6..13): full payment, no reductions
            total_payments += payment_this_bid

        # Apply any accumulated deductions to current payment only if the target is still a non-winner
        # Note: future_interest_shares were only accumulated for bids before the winning bid.
        if bid_idx < TARGET_BID_INDEX - 1:
            deduction = future_interest_shares[bid_idx]
            total_payments -= deduction

    # Interest percent relative to amount received at bid 6
    received_amount = TOTAL_CHIT - discounts[TARGET_BID_INDEX - 1]
    interest_paid = total_payments - received_amount
    interest_percent = (interest_paid / received_amount) * 100
    return interest_percent


def optimize_d6(
    d6_min: int = 7000,
    d6_max: int = 10000,
    trials_per_d6: int = 500,
    seed: int | None = 42,
):
    """
    Sweep D6 discount from d6_min to d6_max and Monte Carlo average interest percent
    across trials. Return the best D6 and a summary.
    """
    if seed is not None:
        random.seed(seed)
    results = []
    for d6 in range(d6_min, d6_max + 1):
        # Monte Carlo across randomized bids 1..5
        samples = [
            simulate_once(d6, seed=random.randint(0, 10**9))
            for _ in range(trials_per_d6)
        ]
        avg_interest = mean(samples)
        results.append((d6, avg_interest))

    # Find minimum average interest percent
    best_d6, best_interest = min(results, key=lambda x: x[1])
    return best_d6, best_interest, results


def main():
    best_d6, best_interest, results = optimize_d6()
    print(f"Best 6th-bid discount: {best_d6} rupees")
    print(f"Minimized average interest: {best_interest:.3f}%")
    print("\nSample of results (discount -> avg interest %):")
    for d6, avg in results[:: max(1, len(results) // 10)]:
        print(f"{d6} -> {avg:.3f}%")


# if __name__ == "__main__":
main()


Best 6th-bid discount: 7005 rupees
Minimized average interest: -0.022%

Sample of results (discount -> avg interest %):
7000 -> -0.020%
7300 -> 0.026%
7600 -> 0.068%
7900 -> 0.111%
8200 -> 0.155%
8500 -> 0.198%
8800 -> 0.238%
9100 -> 0.285%
9400 -> 0.326%
9700 -> 0.372%
10000 -> 0.414%
